L'objectif de ce carnet est de référencer différentes techniques d'aspiration de données sur le web. Elle comprendra notamment :

 - des éléments structurés comme **des listes ou des tableaux HTML**
 - l'utilisation **d'expressions régulières** pour affiner les différentes sélections
 
Les modules **requests et BeautifulSoup** seront allègrement utilisés, assez pour justifier leur appel avant le début des festivités :

In [17]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import requests, re, csv
from bs4 import BeautifulSoup

## Aspirer différents éléments d'un tableau HTML

### Sur une seule page

Nous allons commencer par aspirer **des données tirées de Wikipedia**. L'encyclopédie en ligne est de toute évidence un bon point de départ, car il n'y a aucun doute sur la possibilité **d'aspirer les informations renseignées dessus**.

Un exemple intéressant serait de travailler sur un tableau, par exemple celui recensant **<a href="https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Nancy" target="_blank"> les monuments historiques de Nancy</a>**. L'enjeu va se résumer à : 
- faire des sélections
- les mémoriser en autant de variables
- créer une BDD grâce à elle 
- exporter la BDD finale dans un certain format

Certains services en ligne comme Google Spreadsheet incluent **des formules dédiées**. Par exemple, voici le résultat d'une formule IMPORTHTML() centrée sur le premier tableau de la page Wikipédia précédemment citée :

![IMPORTHTML() sur Google Spreadsheet](illustrations/importhtml_gspreadsheet.png)

Le résultat est propre, mais :

 - les coordonnées géographiques **sont inexploitables en l'état**. Il serait préférable de séparer longitude et latitude en deux colonnes distinctes
 - on peut faire une remarque équivalente pour les notices. **L'identifiant n'est pas franchement utile**, l'URL cible le serait plus
 
En se penchant sur les premières balises du tableau HTML de la page source, voici ce qu'on obtient :

Deux choses intéressantes à remarquer :

- le lien de classe mw-kartographer-maplink contient les latitude et longitude du lieu en métadonnées
- on peut récupérer directement le lien hypertexte de la notice

Sur ces entrefaites, nous pouvons passer au script :

In [18]:
url = requests.get("https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Nancy")
soupe = BeautifulSoup(url.text, "lxml") # "lxml" n'est pas obligatoire, c'est plus une convention
tableau = soupe.find("table", {"class":"wikitable sortable"}) # on considère le premier tableau de classe "wikitable sortable"
lignes = tableau.findAll("tr") # puis toutes ses lignes

Les lignes précédentes ont transformé en "soupe" le code source de la page Wikipédia interrogée. On a ensuite pu faire nos premières sélections avec le tableau de class "wikitable sortable" et l'ensemble de ses lignes.

On va partir sur un csv classqiue :

In [19]:
ficsv = open('livraisons/monuments_histo_nancy.csv','w+')

On peut à présent **parcourir la variable lignes** via une boucle for. A chaque passage dans la boucle, on vérifie l'existence des informations qui nous intéresse, puis on les stocke dans une nouvelle ligne du csv.

En Python, cela se traduit ainsi :

In [20]:
try:
	majcsv = csv.writer(ficsv) # on passe notre csv en mode "écriture"
	majcsv.writerow(('Monument','Adresse','Longitude','Latitude','Source')) # on paramètre nos noms de colonnes
	for ligne in lignes:
		info_monument = ligne.findAll("td") # on fait les premières sélections
		carto = ligne.find("a", {"class":"mw-kartographer-maplink"})
		notice = ligne.findAll("a", {"href":re.compile("^https://www.pop.culture.gouv")}) # ^ signifie qu'on considère le strict début d'une chaîne de caractères
		if info_monument and carto and notice: # si les sélections existent, on enregistre les informations pour le csv
			monument = info_monument[0].get("data-sort-value")
			adresse = info_monument[1].getText()
			longitude = float(carto.get('data-lat'))
			latitude = float(carto.get('data-lon'))
			source = notice[0].get('href')
			majcsv.writerow((monument, adresse, longitude, latitude, source))
finally:   # quand toute la variables lignes a été parcourue, en referme le fichier
	ficsv.close()

Nous pouvons à présent vérifier le contenu du fichier ainsi créé. La bibliothèque pandas est toute désignée :

In [23]:
import pandas as pd

fichier_nancy = pd.read_csv('livraisons/monuments_histo_nancy.csv')

fichier_nancy.head()

,Monument,Adresse,Longitude,Latitude,Source
0,Arc Here,rue Héré\n,48.694416,6.182674,https://www.pop.culture.gouv.fr/notice/merimee...
1,Banque SNVB,4 place André-Maginot\n,48.689889,6.177306,https://www.pop.culture.gouv.fr/notice/merimee...
2,Basilique Saint-Epvre,place Saint-Epvre et place du Colonel-Fabien\n,48.696000,6.180000,https://www.pop.culture.gouv.fr/notice/merimee...
3,Bastion Haussonville,1 rue Gustave-Simon\n,48.693800,6.181900,https://www.pop.culture.gouv.fr/notice/merimee...
4,Brasserie Excelsior,1 rue Mazagranrue Henri-Poincaré\n,48.690833,6.175694,https://www.pop.culture.gouv.fr/notice/merimee...


Tout est propre, on peut passer à l'étape supérieure !

### Sur plusieurs pages

Un autre avantage de coder ses propres scrapers est de pouvoir **automatiser un enchaîne efficace sur plusieurs pages**.

Par exemple, sur les monuments historiques renseignés sur Wikipedia, le seul changement concerne l'URL. La page de Strasbourg se trouve à l'adresse https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Strasbourg, celle de Mulhouse à l'adresse https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Mulhouse, etc...

Bref, on peut stocker les différentes fins d'URL dans une variable, et parcourir **cette dernière dans une boucle for**. A chaque passage, **on scelle une nouvelle URL** et on reproduit ce qui a été éprouvé avant :

In [22]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import requests, re, csv
from bs4 import BeautifulSoup

depart ="https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_d"
villes = ["e_Reims", "e_Chaumont", "e_Charleville-Mézières", "e_Toul", "e_Pont-à-Mousson", "e_Haguenau", "e_Châlons-en-Champagne", "e_Troyes", "e_Verdun", "e_Bar-le-Duc", "'Épinal", "e_Nancy", "e_Metz", "e_Colmar", "e_Sélestat", "e_Mulhouse", "e_Strasbourg"]
ficsv = open('livraisons/monuments_histo_ge.csv','w+')

try:
	majcsv = csv.writer(ficsv)
	majcsv.writerow(('Monument','Adresse','Longitude','Latitude','Source'))
	for ville in villes:
		url_ville = depart+ville # une simple addition de chaînes de cara' nous donne nos URL complètes
		url = requests.get(url_ville) # et on ouvre l'URL actuelle
		print("Lien : "+url_ville)
		soupe = BeautifulSoup(url.text, "lxml")
		tableau = soupe.find("table", {"class":"wikitable sortable"})
		lignes = tableau.findAll("tr")
		for ligne in lignes:
			monuments = ligne.findAll("td")
			carto = ligne.find("a", {"class":"mw-kartographer-maplink"})
			source = ligne.findAll("a", {"href":re.compile("^https://www.pop.culture.gouv")}) 
			if monuments and carto and source:
				monument = monuments[0].get("data-sort-value")
				adresse = monuments[1].get_text()
				longitude = float(carto.get('data-lat'))
				latitude = float(carto.get('data-lon'))
				lien = source[0].get('href')
				majcsv.writerow((monument, adresse, longitude, latitude, lien))
finally:
	ficsv.close()

Lien : https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Reims
Lien : https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Chaumont
Lien : https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Charleville-Mézières
Lien : https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Toul
Lien : https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Pont-à-Mousson
Lien : https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Haguenau
Lien : https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Châlons-en-Champagne
Lien : https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Troyes
Lien : https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Verdun
Lien : https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Bar-le-Duc
Lien : https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_d'Épinal
Lien : https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Nancy
Lien : https:

Si on reproduit les vérifications sur le nouveau fichier, on devrait avoir des monuments rémois en tête et des adresses strasbourgeoises à la fin (l'ordre des villes déclarées dans notre script).

In [24]:
fichier_ge = pd.read_csv('livraisons/monuments_histo_ge.csv')

fichier_ge.head()

,Monument,Adresse,Longitude,Latitude,Source
0,Abbaye Saint-Remi,53 rue Simon\n,49.243400,4.041300,https://www.pop.culture.gouv.fr/notice/merimee...
1,Circuit de Reims-Gueux,\n,49.254167,3.930833,https://www.pop.culture.gouv.fr/notice/merimee...
2,Ancienne Douane,4 place RoyaleRue du CloîtreRue du Grand-Credo\n,49.258000,4.031700,https://www.pop.culture.gouv.fr/notice/merimee...
3,Basilique Saint-Remi,55 rue Simon\n,49.243100,4.041900,https://www.pop.culture.gouv.fr/notice/merimee...
4,Bibliotheque Carnegie,2 place Carnégie\n,49.253000,4.035400,https://www.pop.culture.gouv.fr/notice/merimee...


In [25]:
fichier_ge.tail()

,Monument,Adresse,Longitude,Latitude,Source
1196,Tour Romaine,"47-49, rue des Grandes-Arcades\n",48.582655,7.747430,https://www.pop.culture.gouv.fr/notice/merimee...
1197,Usine Junkers Flugzeug-Und-Motorenwerke,"33, rue du Maréchal Lefebvre\n",48.561528,7.739126,https://www.pop.culture.gouv.fr/notice/merimee...
1198,Villa,"13, rue Fischart\n",48.586920,7.765927,https://www.pop.culture.gouv.fr/notice/merimee...
1199,Villa Faist,"24, rue Twinger\n",48.589036,7.769948,https://www.pop.culture.gouv.fr/notice/merimee...
1200,Villa Stempel,"4, rue Erckmann-Chatrian\n",48.588848,7.764953,https://www.pop.culture.gouv.fr/notice/merimee...


Tout est donc OK.